In [1]:
from rdflib import Graph, URIRef, BNode, Literal, Namespace
from rdflib.namespace import RDF, XSD
from datetime import datetime

# Define custom namespace for humemai ontology
humemai = Namespace("https://humem.ai/ontology/")


class Memory:
    """
    Base Memory class for managing memories of different types.
    Provides methods to add, retrieve, delete, and manage memories in the RDF graph.
    """

    def __init__(self):
        # Initialize RDF graph for memory storage
        self.graph = Graph()
        self.graph.bind("humemai", humemai)

    def add_memory(self, triples: list, qualifiers: dict):
        """
        Add a memory (triples) to the RDF graph, with optional qualifiers.

        Args:
            triples (list): A list of triples (subject, predicate, object) to be added.
            qualifiers (dict): A dictionary of qualifiers (e.g., location, currentTime).
        """
        for subj, pred, obj in triples:
            # Add the main triple [subject, predicate, object]
            self.graph.add((subj, pred, obj))

            # Reify the triple to attach qualifiers
            statement = BNode()  # Blank node to represent the reified statement
            self.graph.add((statement, RDF.type, RDF.Statement))
            self.graph.add((statement, RDF.subject, subj))
            self.graph.add((statement, RDF.predicate, pred))
            self.graph.add((statement, RDF.object, obj))

            # Add provided qualifiers with the correct data types
            for key, value in qualifiers.items():
                if key in [
                    "https://humem.ai/ontology/currentTime",
                    "https://humem.ai/ontology/timestamp",
                ]:
                    # Qualifiers related to time should be xsd:dateTime
                    self.graph.add(
                        (statement, URIRef(key), Literal(value, datatype=XSD.dateTime))
                    )
                elif key == "https://humem.ai/ontology/strength":
                    # 'strength' should be xsd:integer
                    self.graph.add(
                        (statement, URIRef(key), Literal(value, datatype=XSD.integer))
                    )
                else:
                    # Store string literals (like location, emotion, derivedFrom) as plain literals without datatype
                    self.graph.add((statement, URIRef(key), Literal(value)))

    def delete_memory(self, subject: URIRef, predicate: URIRef, object_: URIRef):
        """
        Delete a memory from the RDF graph, including its qualifiers.

        Args:
            subject (URIRef): The subject of the memory triple.
            predicate (URIRef): The predicate of the memory triple.
            object_ (URIRef): The object of the memory triple.
        """
        # Remove the main triple
        self.graph.remove((subject, predicate, object_))

        # Find all reified statements for this triple
        for statement in self.graph.subjects(RDF.type, RDF.Statement):
            s = self.graph.value(statement, RDF.subject)
            p = self.graph.value(statement, RDF.predicate)
            o = self.graph.value(statement, RDF.object)
            if s == subject and p == predicate and o == object_:
                # Remove all triples related to this statement
                for _, _, obj in list(self.graph.triples((statement, None, None))):
                    self.graph.remove((statement, _, obj))

    def get_memories(
        self,
        subject: URIRef = None,
        predicate: URIRef = None,
        object_: URIRef = None,
        location: str = None,
        emotion: str = None,
        derived_from: str = None,
        strength: int = None,
        current_time_lower: str = None,
        current_time_upper: str = None,
        timestamp_lower: str = None,
        timestamp_upper: str = None,
    ) -> "Memory":
        """
        Retrieve memories with optional filtering on subject, predicate, object, location, emotion,
        derivedFrom, and strength. `currentTime` and `timestamp` are provided as intervals.

        Args:
            subject (URIRef, optional): Filter by subject.
            predicate (URIRef, optional): Filter by predicate.
            object_ (URIRef, optional): Filter by object.
            location (str, optional): Filter by location.
            emotion (str, optional): Filter by emotion.
            derived_from (str, optional): Filter by derivedFrom.
            strength (int, optional): Filter by strength.
            current_time_lower (str, optional): Lower bound for currentTime interval.
            current_time_upper (str, optional): Upper bound for currentTime interval.
            timestamp_lower (str, optional): Lower bound for timestamp interval.
            timestamp_upper (str, optional): Upper bound for timestamp interval.

        Returns:
            Memory: A new Memory object containing the filtered memories.
        """
        # Initialize a new Memory object to store the filtered memories
        filtered_memory = self.__class__()

        # SPARQL query to retrieve memories with optional filters
        query = """
        PREFIX humemai: <https://humem.ai/ontology/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

        SELECT ?statement ?subject ?predicate ?object
        WHERE {
            ?statement rdf:type rdf:Statement ;
                       rdf:subject ?subject ;
                       rdf:predicate ?predicate ;
                       rdf:object ?object .
        """

        # Add filters for subject, predicate, and object if provided
        if subject:
            query += f"FILTER(?subject = <{subject}>) .\n"
        if predicate:
            query += f"FILTER(?predicate = <{predicate}>) .\n"
        if object_:
            query += f"FILTER(?object = <{object_}>) .\n"

        # Add filters for qualifiers (location, emotion, derived_from, strength)
        if location:
            query += f'?statement humemai:location "{location}" .\n'
        if emotion:
            query += f'?statement humemai:emotion "{emotion}" .\n'
        if derived_from:
            query += f'?statement humemai:derivedFrom "{derived_from}" .\n'
        if strength is not None:
            query += f"?statement humemai:strength {strength} .\n"

        # Add filters for current_time range (interval)
        if current_time_lower and current_time_upper:
            query += f"""
            ?statement humemai:currentTime ?currentTime .
            FILTER(?currentTime >= "{current_time_lower}"^^xsd:dateTime && ?currentTime <= "{current_time_upper}"^^xsd:dateTime) .
            """

        # Add filters for timestamp range (interval)
        if timestamp_lower and timestamp_upper:
            query += f"""
            ?statement humemai:timestamp ?timestamp .
            FILTER(?timestamp >= "{timestamp_lower}"^^xsd:dateTime && ?timestamp <= "{timestamp_upper}"^^xsd:dateTime) .
            """

        # Close the WHERE block
        query += "}"

        # Execute the SPARQL query
        results = self.graph.query(query)

        # Iterate through the results and add them to the filtered memory graph
        for row in results:
            filtered_memory.graph.add((row.subject, row.predicate, row.object))

            # Reify the statement to attach all the qualifiers
            for qualifier_pred, qualifier_obj in self.graph.predicate_objects(row.statement):
                filtered_memory.graph.add((row.statement, qualifier_pred, qualifier_obj))

        return filtered_memory

    def get_memory_count(self) -> int:
        """
        Count the number of unique memories (subject-predicate-object triples) in the graph.

        Returns:
            int: The count of unique memories.
        """
        unique_memories = set()

        # Iterate over reified statements and extract subject-predicate-object triples
        for s, p, o in self.graph.triples((None, RDF.type, RDF.Statement)):
            subj = self.graph.value(s, RDF.subject)
            pred = self.graph.value(s, RDF.predicate)
            obj = self.graph.value(s, RDF.object)

            unique_memories.add((subj, pred, obj))

        return len(unique_memories)

    def __repr__(self) -> str:
        """
        Provide a readable representation of the Memory object.
        Display triples and their qualifiers in a more readable format.
        """
        memories = {}
        for s, p, o in self.graph.triples((None, RDF.type, RDF.Statement)):
            subj = self.graph.value(s, RDF.subject)
            pred = self.graph.value(s, RDF.predicate)
            obj = self.graph.value(s, RDF.object)

            triple_key = (subj, pred, obj)

            # Initialize memory for this triple if not already done
            if triple_key not in memories:
                memories[triple_key] = {}

            # Gather all the qualifiers for this triple
            for q_pred, q_obj in self.graph.predicate_objects(s):
                if q_pred not in (RDF.type, RDF.subject, RDF.predicate, RDF.object):
                    memories[triple_key][str(q_pred)] = str(q_obj)

        # Create a readable format for output
        memory_strings = [
            f"[{subj}, {pred}, {obj}, {qualifiers}]"
            for (subj, pred, obj), qualifiers in memories.items()
        ]

        return "\n".join(memory_strings)


class ShortTerm(Memory):
    """
    ShortTerm memory class that inherits from Memory.
    Handles specific short-term memory functionalities.
    """

    def add_short_term_memory(
        self, triples: list, location: str, current_time: str = None
    ):
        """
        Add short-term memories to the RDF graph, with qualifiers for 'currentTime' and 'location'.

        Args:
            triples (list): A list of triples (subject, predicate, object) to be stored.
            location (str): The location related to the memories.
            current_time (str): The current time of the memories (default is the current timestamp).
        """
        if current_time is None:
            current_time = datetime.now().isoformat()

        qualifiers = {
            "https://humem.ai/ontology/location": location,
            "https://humem.ai/ontology/currentTime": current_time,
        }

        self.add_memory(triples, qualifiers)

    def get_memories(
        self,
        subject: URIRef = None,
        predicate: URIRef = None,
        object_: URIRef = None,
        location: str = None,
        current_time_lower: str = None,
        current_time_upper: str = None,
    ) -> "Memory":
        """
        Retrieve all short-term memories (those that contain the 'currentTime' qualifier).
        Optionally filter by subject, predicate, object, location, or a range of 'currentTime'.

        Args:
            subject (URIRef, optional): Filter by subject.
            predicate (URIRef, optional): Filter by predicate.
            object_ (URIRef, optional): Filter by object.
            location (str, optional): Filter by location.
            current_time_lower (str, optional): Lower bound for currentTime interval.
            current_time_upper (str, optional): Upper bound for currentTime interval.

        Returns:
            Memory: A new Memory object containing the filtered short-term memories.
        """
        return super().get_memories(
            subject=subject,
            predicate=predicate,
            object_=object_,
            location=location,
            current_time_lower=current_time_lower,
            current_time_upper=current_time_upper,
        )


class LongTerm(Memory):
    """
    LongTerm memory class that inherits from Memory.
    Handles specific long-term memory functionalities.
    """

    def add_long_term_memory(
        self,
        triples: list,
        location: str = None,
        timestamp: str = None,
        emotion: str = None,
        derived_from: str = None,
        strength: int = None,
    ):
        """
        Add long-term memories to the RDF graph, with optional qualifiers for 'location',
        'timestamp', 'emotion', 'derivedFrom', and 'strength'.

        Args:
            triples (list): A list of triples (subject, predicate, object) to be stored.
            location (str, optional): The location related to the memories.
            timestamp (str, optional): The timestamp of the memories in ISO format.
            emotion (str, optional): The emotional state associated with the memories.
            derived_from (str, optional): The source or event from which the memory is derived.
            strength (int, optional): The strength of the memory (an integer value).
        """
        qualifiers = {}

        if location:
            qualifiers["https://humem.ai/ontology/location"] = location
        if timestamp:
            qualifiers["https://humem.ai/ontology/timestamp"] = timestamp
        if emotion:
            qualifiers["https://humem.ai/ontology/emotion"] = emotion
        if derived_from:
            qualifiers["https://humem.ai/ontology/derivedFrom"] = derived_from
        if strength is not None:
            qualifiers["https://humem.ai/ontology/strength"] = strength

        self.add_memory(triples, qualifiers)

    def get_memories(
        self,
        subject: URIRef = None,
        predicate: URIRef = None,
        object_: URIRef = None,
        location: str = None,
        emotion: str = None,
        derived_from: str = None,
        strength: int = None,
        timestamp_lower: str = None,
        timestamp_upper: str = None,
    ) -> "Memory":
        """
        Retrieve long-term memories that have at least one of the qualifiers: 'location', 'emotion',
        'timestamp', 'derivedFrom', or 'strength', and exclude those with the 'currentTime' qualifier.
        Optionally filter by these qualifiers and by a range of 'timestamp'.

        Args:
            subject (URIRef, optional): Filter by subject.
            predicate (URIRef, optional): Filter by predicate.
            object_ (URIRef, optional): Filter by object.
            location (str, optional): Filter by location.
            emotion (str, optional): Filter by emotion.
            derived_from (str, optional): Filter by derivedFrom.
            strength (int, optional): Filter by strength.
            timestamp_lower (str, optional): Lower bound for timestamp interval.
            timestamp_upper (str, optional): Upper bound for timestamp interval.

        Returns:
            Memory: A new Memory object containing all long-term memories.
        """
        return super().get_memories(
            subject=subject,
            predicate=predicate,
            object_=object_,
            location=location,
            emotion=emotion,
            derived_from=derived_from,
            strength=strength,
            timestamp_lower=timestamp_lower,
            timestamp_upper=timestamp_upper,
        )


class MemorySystem:
    """
    MemorySystem class that encapsulates both short-term and long-term memories.
    Provides methods to interact with the overall memory system.
    """

    def __init__(self):
        self.short_term_memory = ShortTerm()
        self.long_term_memory = LongTerm()

    def get_working_memory(self) -> "Memory":
        """
        Retrieve working memory, which is a combination of short-term memories and
        partial long-term memories. The criteria for partial long-term memory can be
        defined as needed.

        Returns:
            Memory: A new Memory object containing the working memory.
        """
        working_memory = Memory()

        # Retrieve all short-term memories
        short_term = self.short_term_memory.get_memories()
        for s, p, o in short_term.graph:
            working_memory.graph.add((s, p, o))
        for s, p, o in short_term.graph.triples((None, RDF.type, RDF.Statement)):
            for qualifier_pred, qualifier_obj in short_term.graph.predicate_objects(s):
                if p != RDF.type and p != RDF.subject and p != RDF.predicate and p != RDF.object:
                    working_memory.graph.add((s, qualifier_pred, qualifier_obj))

        # Retrieve partial long-term memories
        # Placeholder: Modify this section to determine partial long-term memories as needed
        # For demonstration, let's assume partial long-term memories are those with strength >= 8
        partial_long_term = self.long_term_memory.get_memories(strength=8)
        for s, p, o in partial_long_term.graph:
            working_memory.graph.add((s, p, o))
        for s, p, o in partial_long_term.graph.triples((None, RDF.type, RDF.Statement)):
            for qualifier_pred, qualifier_obj in partial_long_term.graph.predicate_objects(s):
                if p != RDF.type and p != RDF.subject and p != RDF.predicate and p != RDF.object:
                    working_memory.graph.add((s, qualifier_pred, qualifier_obj))

        return working_memory

    def __repr__(self) -> str:
        """
        Provide a readable representation of the MemorySystem object.
        Displays both short-term and long-term memories.
        """
        return f"MemorySystem(\nShort-Term Memories:\n{self.short_term_memory}\n\nLong-Term Memories:\n{self.long_term_memory}\n)"


# Example Usage
if __name__ == "__main__":
    from rdflib import URIRef

    # Initialize the MemorySystem
    memory_system = MemorySystem()

    # Define some example triples (subject, predicate, object) for both short-term and long-term memories
    triples1 = [
        (
            URIRef("https://humem.ai/people/PersonA"),
            URIRef("https://humem.ai/ontology/likes"),
            URIRef("https://humem.ai/objects/Book"),
        ),
        (
            URIRef("https://humem.ai/people/PersonA"),
            URIRef("https://humem.ai/ontology/visits"),
            URIRef("https://humem.ai/places/Library"),
        ),
    ]

    triples2 = [
        (
            URIRef("https://humem.ai/people/PersonB"),
            URIRef("https://humem.ai/ontology/visits"),
            URIRef("https://humem.ai/places/Cafe"),
        ),
        (
            URIRef("https://humem.ai/people/PersonB"),
            URIRef("https://humem.ai/ontology/orders"),
            URIRef("https://humem.ai/objects/Coffee"),
        ),
    ]

    triples3 = [
        (
            URIRef("https://humem.ai/people/PersonC"),
            URIRef("https://humem.ai/ontology/reads"),
            URIRef("https://humem.ai/objects/Book"),
        ),
        (
            URIRef("https://humem.ai/people/PersonC"),
            URIRef("https://humem.ai/ontology/worksAt"),
            URIRef("https://humem.ai/places/Office"),
        ),
    ]

    triples4 = [
        (
            URIRef("https://humem.ai/people/PersonD"),
            URIRef("https://humem.ai/ontology/knows"),
            URIRef("https://humem.ai/people/PersonA"),
        ),
        (
            URIRef("https://humem.ai/people/PersonD"),
            URIRef("https://humem.ai/ontology/livesIn"),
            URIRef("https://humem.ai/places/City"),
        ),
    ]

    # Add short-term memories (with currentTime and location qualifiers)
    memory_system.short_term_memory.add_short_term_memory(
        triples1, location="Library", current_time="2024-09-30T10:00:00"
    )
    memory_system.short_term_memory.add_short_term_memory(
        triples2, location="Cafe", current_time="2024-09-30T11:30:00"
    )
    memory_system.short_term_memory.add_short_term_memory(
        triples3, location="Office", current_time="2024-09-30T12:00:00"
    )

    # Add long-term memories (with optional qualifiers)
    memory_system.long_term_memory.add_long_term_memory(
        triples4,
        location="City",
        timestamp="2024-08-15T09:00:00",
        emotion="Happy",
        derived_from="https://humem.ai/events/Event1",
        strength=10,
    )
    memory_system.long_term_memory.add_long_term_memory(
        triples2,
        location="Cafe",
        timestamp="2024-06-10T14:30:00",
        emotion="Content",
        derived_from="https://humem.ai/events/Event2",
        strength=8,
    )

    # Function to display memories
    def display_memories(memory_obj, title):
        print(f"\n{title}:")
        print(memory_obj)
        print(f"Total Memories: {memory_obj.get_memory_count()}")

    # Display all short-term memories
    display_memories(memory_system.short_term_memory, "All Short-Term Memories")

    # Display all long-term memories
    display_memories(memory_system.long_term_memory, "All Long-Term Memories")

    # Retrieve working memory (short-term + partial long-term)
    working_memory = memory_system.get_working_memory()
    display_memories(working_memory, "Working Memory")

    # Example: Delete a specific short-term memory (PersonB visiting Cafe)
    memory_system.short_term_memory.delete_memory(
        subject=URIRef("https://humem.ai/people/PersonB"),
        predicate=URIRef("https://humem.ai/ontology/visits"),
        object_=URIRef("https://humem.ai/places/Cafe"),
    )

    # Delete a specific long-term memory (PersonB visiting Cafe with strength 8)
    memory_system.long_term_memory.delete_memory(
        subject=URIRef("https://humem.ai/people/PersonB"),
        predicate=URIRef("https://humem.ai/ontology/visits"),
        object_=URIRef("https://humem.ai/places/Cafe"),
    )

    # Display memories after deletion
    display_memories(
        memory_system.short_term_memory, "Short-Term Memories After Deletion"
    )
    display_memories(
        memory_system.long_term_memory, "Long-Term Memories After Deletion"
    )

    # Retrieve working memory after deletion
    working_memory_after_deletion = memory_system.get_working_memory()
    display_memories(
        working_memory_after_deletion, "Working Memory After Deletion"
    )



All Short-Term Memories:
[https://humem.ai/people/PersonA, https://humem.ai/ontology/likes, https://humem.ai/objects/Book, {'https://humem.ai/ontology/location': 'Library', 'https://humem.ai/ontology/currentTime': '2024-09-30T10:00:00'}]
[https://humem.ai/people/PersonA, https://humem.ai/ontology/visits, https://humem.ai/places/Library, {'https://humem.ai/ontology/location': 'Library', 'https://humem.ai/ontology/currentTime': '2024-09-30T10:00:00'}]
[https://humem.ai/people/PersonB, https://humem.ai/ontology/visits, https://humem.ai/places/Cafe, {'https://humem.ai/ontology/location': 'Cafe', 'https://humem.ai/ontology/currentTime': '2024-09-30T11:30:00'}]
[https://humem.ai/people/PersonB, https://humem.ai/ontology/orders, https://humem.ai/objects/Coffee, {'https://humem.ai/ontology/location': 'Cafe', 'https://humem.ai/ontology/currentTime': '2024-09-30T11:30:00'}]
[https://humem.ai/people/PersonC, https://humem.ai/ontology/reads, https://humem.ai/objects/Book, {'https://humem.ai/ontol

In [2]:
# Define the Memory classes and methods
# (Use the previous implementation of ShortTerm and LongTerm classes)

# Initialize short-term and long-term memory objects
short_term_memory = ShortTerm()
long_term_memory = LongTerm()

# Define some example triples (subject, predicate, object) for both short-term and long-term memories
triples1 = [
    (URIRef("https://humem.ai/people/PersonA"), URIRef("https://humem.ai/ontology/likes"), URIRef("https://humem.ai/objects/Book")),
    (URIRef("https://humem.ai/people/PersonA"), URIRef("https://humem.ai/ontology/visits"), URIRef("https://humem.ai/places/Library")),
]

triples2 = [
    (URIRef("https://humem.ai/people/PersonB"), URIRef("https://humem.ai/ontology/visits"), URIRef("https://humem.ai/places/Cafe")),
    (URIRef("https://humem.ai/people/PersonB"), URIRef("https://humem.ai/ontology/orders"), URIRef("https://humem.ai/objects/Coffee")),
]

triples3 = [
    (URIRef("https://humem.ai/people/PersonC"), URIRef("https://humem.ai/ontology/reads"), URIRef("https://humem.ai/objects/Book")),
    (URIRef("https://humem.ai/people/PersonC"), URIRef("https://humem.ai/ontology/worksAt"), URIRef("https://humem.ai/places/Office")),
]

triples4 = [
    (URIRef("https://humem.ai/people/PersonD"), URIRef("https://humem.ai/ontology/knows"), URIRef("https://humem.ai/people/PersonA")),
    (URIRef("https://humem.ai/people/PersonD"), URIRef("https://humem.ai/ontology/livesIn"), URIRef("https://humem.ai/places/City")),
]

# Add short-term memories (with currentTime and location qualifiers)
short_term_memory.add_short_term_memory(triples1, location="Library", current_time="2024-09-30T10:00:00")
short_term_memory.add_short_term_memory(triples2, location="Cafe", current_time="2024-09-30T11:30:00")
short_term_memory.add_short_term_memory(triples3, location="Office", current_time="2024-09-30T12:00:00")

# Add long-term memories (with optional qualifiers)
long_term_memory.add_long_term_memory(triples4, location="City", timestamp="2024-08-15T09:00:00", emotion="Happy", derived_from="https://humem.ai/events/Event1", strength=10)
long_term_memory.add_long_term_memory(triples2, location="Cafe", timestamp="2024-07-10T14:30:00", emotion="Content", derived_from="https://humem.ai/events/Event2", strength=8)

# Example 1: Retrieve all short-term memories with no filters
print("\nExample 1: All Short-Term Memories")
filtered_short_term_memory_1 = short_term_memory.get_memories()
print(filtered_short_term_memory_1)

# Example 2: Retrieve all long-term memories with no filters
print("\nExample 2: All Long-Term Memories")
filtered_long_term_memory_2 = long_term_memory.get_memories()
print(filtered_long_term_memory_2)

# Example 3: Retrieve short-term memories where the subject is PersonA
print("\nExample 3: Short-Term Memories for PersonA")
filtered_short_term_memory_3 = short_term_memory.get_memories(subject=URIRef("https://humem.ai/people/PersonA"))
print(filtered_short_term_memory_3)

# Example 4: Retrieve short-term memories where the location is "Cafe"
print("\nExample 4: Short-Term Memories in Cafe")
filtered_short_term_memory_4 = short_term_memory.get_memories(location="Cafe")
print(filtered_short_term_memory_4)

# Example 5: Retrieve long-term memories where the object is "PersonA"
print("\nExample 5: Long-Term Memories with object PersonA")
filtered_long_term_memory_5 = long_term_memory.get_memories(object_=URIRef("https://humem.ai/people/PersonA"))
print(filtered_long_term_memory_5)

# Example 6: Retrieve long-term memories in a time range
print("\nExample 6: Long-Term Memories between 2024-07-01 and 2024-08-31")
filtered_long_term_memory_6 = long_term_memory.get_memories(
    timestamp_lower="2024-07-01T00:00:00",
    timestamp_upper="2024-08-31T23:59:59"
)
print(filtered_long_term_memory_6)

# Example 7: Retrieve short-term memories in a specific currentTime range
print("\nExample 7: Short-Term Memories between 2024-09-30T10:00:00 and 2024-09-30T12:00:00")
filtered_short_term_memory_7 = short_term_memory.get_memories(
    current_time_lower="2024-09-30T10:00:00",
    current_time_upper="2024-09-30T12:00:00"
)
print(filtered_short_term_memory_7)

# Example 8: Retrieve long-term memories where the location is "Cafe" and strength is 8
print("\nExample 8: Long-Term Memories in Cafe with Strength 8")
filtered_long_term_memory_8 = long_term_memory.get_memories(location="Cafe", strength=8)
print(filtered_long_term_memory_8)

# Example 9: Retrieve long-term memories where the emotion is "Happy"
print("\nExample 9: Long-Term Memories with Emotion 'Happy'")
filtered_long_term_memory_9 = long_term_memory.get_memories(emotion="Happy")
print(filtered_long_term_memory_9)

# Example 10: Retrieve short-term memories with specific predicate
print("\nExample 10: Short-Term Memories where the predicate is 'visits'")
filtered_short_term_memory_10 = short_term_memory.get_memories(predicate=URIRef("https://humem.ai/ontology/visits"))
print(filtered_short_term_memory_10)




Example 1: All Short-Term Memories
[https://humem.ai/people/PersonA, https://humem.ai/ontology/likes, https://humem.ai/objects/Book, {'https://humem.ai/ontology/location': 'Library', 'https://humem.ai/ontology/currentTime': '2024-09-30T10:00:00'}]
[https://humem.ai/people/PersonA, https://humem.ai/ontology/visits, https://humem.ai/places/Library, {'https://humem.ai/ontology/location': 'Library', 'https://humem.ai/ontology/currentTime': '2024-09-30T10:00:00'}]
[https://humem.ai/people/PersonB, https://humem.ai/ontology/visits, https://humem.ai/places/Cafe, {'https://humem.ai/ontology/location': 'Cafe', 'https://humem.ai/ontology/currentTime': '2024-09-30T11:30:00'}]
[https://humem.ai/people/PersonB, https://humem.ai/ontology/orders, https://humem.ai/objects/Coffee, {'https://humem.ai/ontology/location': 'Cafe', 'https://humem.ai/ontology/currentTime': '2024-09-30T11:30:00'}]
[https://humem.ai/people/PersonC, https://humem.ai/ontology/reads, https://humem.ai/objects/Book, {'https://hume

In [3]:
from rdflib import URIRef

# Initialize short-term and long-term memory objects
short_term_memory = ShortTerm()
long_term_memory = LongTerm()

# Define some example triples (subject, predicate, object) for both short-term and long-term memories
triples1 = [
    (URIRef("https://humem.ai/people/PersonA"), URIRef("https://humem.ai/ontology/likes"), URIRef("https://humem.ai/objects/Book")),
    (URIRef("https://humem.ai/people/PersonA"), URIRef("https://humem.ai/ontology/visits"), URIRef("https://humem.ai/places/Library")),
]

triples2 = [
    (URIRef("https://humem.ai/people/PersonB"), URIRef("https://humem.ai/ontology/visits"), URIRef("https://humem.ai/places/Cafe")),
    (URIRef("https://humem.ai/people/PersonB"), URIRef("https://humem.ai/ontology/orders"), URIRef("https://humem.ai/objects/Coffee")),
]

triples3 = [
    (URIRef("https://humem.ai/people/PersonC"), URIRef("https://humem.ai/ontology/reads"), URIRef("https://humem.ai/objects/Book")),
    (URIRef("https://humem.ai/people/PersonC"), URIRef("https://humem.ai/ontology/worksAt"), URIRef("https://humem.ai/places/Office")),
]

triples4 = [
    (URIRef("https://humem.ai/people/PersonD"), URIRef("https://humem.ai/ontology/knows"), URIRef("https://humem.ai/people/PersonA")),
    (URIRef("https://humem.ai/people/PersonD"), URIRef("https://humem.ai/ontology/livesIn"), URIRef("https://humem.ai/places/City")),
]

# Add short-term memories (with currentTime and location qualifiers)
short_term_memory.add_short_term_memory(triples1, location="Library", current_time="2024-09-30T10:00:00")
short_term_memory.add_short_term_memory(triples2, location="Cafe", current_time="2024-09-30T11:30:00")
short_term_memory.add_short_term_memory(triples3, location="Office", current_time="2024-09-30T12:00:00")

# Add long-term memories (with optional qualifiers)
long_term_memory.add_long_term_memory(triples4, location="City", timestamp="2024-08-15T09:00:00", emotion="Happy", derived_from="https://humem.ai/events/Event1", strength=10)
long_term_memory.add_long_term_memory(triples2, location="Cafe", timestamp="2024-06-10T14:30:00", emotion="Content", derived_from="https://humem.ai/events/Event2", strength=8)

# Function to display memories
def display_memories(memory_obj, title):
    print(f"\n{title}:")
    print(memory_obj)
    print(f"Total Memories: {memory_obj.get_memory_count()}")

# Display all short-term memories
display_memories(short_term_memory, "All Short-Term Memories")

# Display all long-term memories
display_memories(long_term_memory, "All Long-Term Memories")

# Delete a specific short-term memory (PersonB visiting Cafe)
short_term_memory.delete_memory(
    subject=URIRef("https://humem.ai/people/PersonB"),
    predicate=URIRef("https://humem.ai/ontology/visits"),
    object_=URIRef("https://humem.ai/places/Cafe")
)

# Delete a specific long-term memory (PersonB visiting Cafe with strength 8)
long_term_memory.delete_memory(
    subject=URIRef("https://humem.ai/people/PersonB"),
    predicate=URIRef("https://humem.ai/ontology/visits"),
    object_=URIRef("https://humem.ai/places/Cafe")
)

# Display memories after deletion
display_memories(short_term_memory, "Short-Term Memories After Deletion")
display_memories(long_term_memory, "Long-Term Memories After Deletion")



All Short-Term Memories:
[https://humem.ai/people/PersonA, https://humem.ai/ontology/likes, https://humem.ai/objects/Book, {'https://humem.ai/ontology/location': 'Library', 'https://humem.ai/ontology/currentTime': '2024-09-30T10:00:00'}]
[https://humem.ai/people/PersonA, https://humem.ai/ontology/visits, https://humem.ai/places/Library, {'https://humem.ai/ontology/location': 'Library', 'https://humem.ai/ontology/currentTime': '2024-09-30T10:00:00'}]
[https://humem.ai/people/PersonB, https://humem.ai/ontology/visits, https://humem.ai/places/Cafe, {'https://humem.ai/ontology/location': 'Cafe', 'https://humem.ai/ontology/currentTime': '2024-09-30T11:30:00'}]
[https://humem.ai/people/PersonB, https://humem.ai/ontology/orders, https://humem.ai/objects/Coffee, {'https://humem.ai/ontology/location': 'Cafe', 'https://humem.ai/ontology/currentTime': '2024-09-30T11:30:00'}]
[https://humem.ai/people/PersonC, https://humem.ai/ontology/reads, https://humem.ai/objects/Book, {'https://humem.ai/ontol

In [4]:
# Initialize the MemorySystem
memory_system = MemorySystem()

# Define some example triples (subject, predicate, object) for both short-term and long-term memories
triples1 = [
    (
        URIRef("https://humem.ai/people/PersonA"),
        URIRef("https://humem.ai/ontology/likes"),
        URIRef("https://humem.ai/objects/Book"),
    ),
    (
        URIRef("https://humem.ai/people/PersonA"),
        URIRef("https://humem.ai/ontology/visits"),
        URIRef("https://humem.ai/places/Library"),
    ),
]

triples2 = [
    (
        URIRef("https://humem.ai/people/PersonB"),
        URIRef("https://humem.ai/ontology/visits"),
        URIRef("https://humem.ai/places/Cafe"),
    ),
    (
        URIRef("https://humem.ai/people/PersonB"),
        URIRef("https://humem.ai/ontology/orders"),
        URIRef("https://humem.ai/objects/Coffee"),
    ),
]

triples3 = [
    (
        URIRef("https://humem.ai/people/PersonC"),
        URIRef("https://humem.ai/ontology/reads"),
        URIRef("https://humem.ai/objects/Book"),
    ),
    (
        URIRef("https://humem.ai/people/PersonC"),
        URIRef("https://humem.ai/ontology/worksAt"),
        URIRef("https://humem.ai/places/Office"),
    ),
]

triples4 = [
    (
        URIRef("https://humem.ai/people/PersonD"),
        URIRef("https://humem.ai/ontology/knows"),
        URIRef("https://humem.ai/people/PersonA"),
    ),
    (
        URIRef("https://humem.ai/people/PersonD"),
        URIRef("https://humem.ai/ontology/livesIn"),
        URIRef("https://humem.ai/places/City"),
    ),
]

# Add short-term memories (with currentTime and location qualifiers)
memory_system.short_term_memory.add_short_term_memory(
    triples1, location="Library", current_time="2024-09-30T10:00:00"
)
memory_system.short_term_memory.add_short_term_memory(
    triples2, location="Cafe", current_time="2024-09-30T11:30:00"
)
memory_system.short_term_memory.add_short_term_memory(
    triples3, location="Office", current_time="2024-09-30T12:00:00"
)

# Add long-term memories (with optional qualifiers)
memory_system.long_term_memory.add_long_term_memory(
    triples4,
    location="City",
    timestamp="2024-08-15T09:00:00",
    emotion="Happy",
    derived_from="https://humem.ai/events/Event1",
    strength=10,
)
memory_system.long_term_memory.add_long_term_memory(
    triples2,
    location="Cafe",
    timestamp="2024-06-10T14:30:00",
    emotion="Content",
    derived_from="https://humem.ai/events/Event2",
    strength=8,
)

# Function to display memories
def display_memories(memory_obj, title):
    print(f"\n{title}:")
    print(memory_obj)
    print(f"Total Memories: {memory_obj.get_memory_count()}")

# Display all short-term memories
display_memories(memory_system.short_term_memory, "All Short-Term Memories")

# Display all long-term memories
display_memories(memory_system.long_term_memory, "All Long-Term Memories")

# Retrieve working memory (short-term + partial long-term)
working_memory = memory_system.get_working_memory()
display_memories(working_memory, "Working Memory")

# Example: Delete a specific short-term memory (PersonB visiting Cafe)
memory_system.short_term_memory.delete_memory(
    subject=URIRef("https://humem.ai/people/PersonB"),
    predicate=URIRef("https://humem.ai/ontology/visits"),
    object_=URIRef("https://humem.ai/places/Cafe"),
)

# Delete a specific long-term memory (PersonB visiting Cafe with strength 8)
memory_system.long_term_memory.delete_memory(
    subject=URIRef("https://humem.ai/people/PersonB"),
    predicate=URIRef("https://humem.ai/ontology/visits"),
    object_=URIRef("https://humem.ai/places/Cafe"),
)

# Display memories after deletion
display_memories(
    memory_system.short_term_memory, "Short-Term Memories After Deletion"
)
display_memories(
    memory_system.long_term_memory, "Long-Term Memories After Deletion"
)

# Retrieve working memory after deletion
working_memory_after_deletion = memory_system.get_working_memory()
display_memories(
    working_memory_after_deletion, "Working Memory After Deletion"
)


All Short-Term Memories:
[https://humem.ai/people/PersonA, https://humem.ai/ontology/likes, https://humem.ai/objects/Book, {'https://humem.ai/ontology/location': 'Library', 'https://humem.ai/ontology/currentTime': '2024-09-30T10:00:00'}]
[https://humem.ai/people/PersonA, https://humem.ai/ontology/visits, https://humem.ai/places/Library, {'https://humem.ai/ontology/location': 'Library', 'https://humem.ai/ontology/currentTime': '2024-09-30T10:00:00'}]
[https://humem.ai/people/PersonB, https://humem.ai/ontology/visits, https://humem.ai/places/Cafe, {'https://humem.ai/ontology/location': 'Cafe', 'https://humem.ai/ontology/currentTime': '2024-09-30T11:30:00'}]
[https://humem.ai/people/PersonB, https://humem.ai/ontology/orders, https://humem.ai/objects/Coffee, {'https://humem.ai/ontology/location': 'Cafe', 'https://humem.ai/ontology/currentTime': '2024-09-30T11:30:00'}]
[https://humem.ai/people/PersonC, https://humem.ai/ontology/reads, https://humem.ai/objects/Book, {'https://humem.ai/ontol